# Libraries

In [330]:
import requests

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) # display all columns in DF
import dtale

import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval

# Uploading raw data

In [331]:
df_openFDA_NDC = pd.read_csv('raw_openFDA_NDC_data.csv', sep = '~')
df_medicaid = pd.read_csv('raw_medicaid_data.csv', sep = '~')
df_PPI = pd.read_excel('PPI industry data for Pharmaceutical preparation manufacturing.xlsx', skiprows = 10)
df_atc = pd.read_excel('raw_atc_data.xlsx')

# Data manipulation: Raw data

### OpenFDA NDC

Clearing nda

In [332]:
df_openFDA_NDC['NDA #'] = df_openFDA_NDC['application_number'].str.replace(r'\D', '')
df_openFDA_NDC['NDA #'] = pd.to_numeric(df_openFDA_NDC['NDA #'])

### Medicaid

Clearing labeler and product code

In [333]:
df_medicaid['labeler_code'] = pd.to_numeric(df_medicaid['labeler_code'])
df_medicaid['product_code'] = pd.to_numeric(df_medicaid['product_code'])

Creating a date

In [334]:
df_medicaid['date'] = 'Q' + df_medicaid['quarter'].astype(str) + ' ' + df_medicaid['year'].astype(str)
df_medicaid['date'] = pd.to_datetime(df_medicaid['date'].str.replace(r'(Q\d) (\d+)', r'\2-\1'), errors='coerce')

### PPI

Creating a date

In [335]:
df_PPI['Period'] = df_PPI['Period'].str.replace(r'\D', '')
df_PPI['Period'] = pd.to_numeric(df_PPI['Period'])
df_PPI['day'] = 1
df_PPI['date'] = pd.to_datetime(dict(year = df_PPI['Year'], month = df_PPI['Period'], day= df_PPI['day']))

Calculating PPI multiplier

In [336]:
df_PPI['Value'] = pd.to_numeric(df_PPI['Value'])
df_PPI['PPI_multiplier'] = df_PPI['Value'].iloc[-1] / df_PPI['Value']

# Merging raw data

### Medicaid and PPI

In [337]:
df_medicaid = pd.merge(df_medicaid, df_PPI, on = 'date', how = 'left')

Adjusting for inflation

In [338]:
df_medicaid['total_amount_reimbursed_adj'] = df_medicaid['total_amount_reimbursed'] * df_medicaid['PPI_multiplier']
df_medicaid['price_per_unit_adj'] = df_medicaid['price_per_unit'] * df_medicaid['PPI_multiplier']

Deleting redundant columns

In [339]:
df_medicaid = df_medicaid.drop(columns=['Series ID', 'Year', 'Period', 'Value', 'Unnamed: 4', 'Unnamed: 5', 'day', 'PPI_multiplier'])

In [340]:
len(df_medicaid)

2251895

### OpenFDA and Medicaid

Merging with Medicaid

In [341]:
df_consolidation_data = pd.merge(df_medicaid, df_openFDA_NDC, on = ['labeler_code', 'product_code'], how = 'inner')

In [342]:
len(df_consolidation_data)

549813

### OpenFDA, Medicaid and NLH

Changing the name to capital letters

In [343]:
df_temp = df_atc

In [344]:
df_temp['RxNorm'] = df_atc['RxNorm'].str.upper()

Merging on **name** with orig. data

In [345]:
df_atc_temp = pd.merge(df_consolidation_data['generic_name'], df_temp[['RxNorm', 'Anatomical Therapeutic Chemical (ATC1)', 'ID (ATC1)']], left_on = ['generic_name'], right_on = ['RxNorm'], how = 'right')

Creating dummy variables for ATC

In [346]:
df_dummy_temp = pd.get_dummies(df_atc_temp['Anatomical Therapeutic Chemical (ATC1)'])

Merging with orig. data

In [347]:
df_atc_temp = pd.merge(df_atc_temp, df_dummy_temp, left_index = True, right_index = True, how = 'inner')

Grouping by name to include multiple ATC for each drug 

In [348]:
df_atc_generic_name_temp = df_atc_temp.set_index(['generic_name']).groupby(level = ['generic_name'])[['Alimentary Tract And Metabolism ', 'Antiinfectives For Systemic Use', 'Antineoplastic And Immunomodulating Agents', 'Antiparasitic Products, Insecticides And Repellents', 'Blood And Blood Forming Organs', 'Cardiovascular System', 'Dermatologicals', 'Genito Urinary System And Sex Hormones', 'Musculo-Skeletal System', 'Nervous System', 'Respiratory System', 'Sensory Organs', 'Systemic Hormonal Preparations, Excl. Sex Hormones And Insulins', 'Various' ]].agg('sum')
df_atc_generic_name_temp = df_atc_name_temp.reset_index()

Changing the aggregated count to a dummy value

In [349]:
df_atc_generic_name_temp['Alimentary Tract And Metabolism '] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Alimentary Tract And Metabolism ']]
df_atc_generic_name_temp['Antiinfectives For Systemic Use'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Antiinfectives For Systemic Use']]
df_atc_generic_name_temp['Antineoplastic And Immunomodulating Agents'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Antineoplastic And Immunomodulating Agents']]
df_atc_generic_name_temp['Antiparasitic Products, Insecticides And Repellents'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Antiparasitic Products, Insecticides And Repellents']]
df_atc_generic_name_temp['Blood And Blood Forming Organs'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Blood And Blood Forming Organs']]
df_atc_generic_name_temp['Cardiovascular System'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Cardiovascular System']]
df_atc_generic_name_temp['Dermatologicals'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Dermatologicals']]
df_atc_generic_name_temp['Genito Urinary System And Sex Hormones'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Genito Urinary System And Sex Hormones']]
df_atc_generic_name_temp['Musculo-Skeletal System'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Musculo-Skeletal System']]
df_atc_generic_name_temp['Nervous System'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Nervous System']]
df_atc_generic_name_temp['Respiratory System'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Respiratory System']]
df_atc_generic_name_temp['Sensory Organs'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Sensory Organs']]
df_atc_generic_name_temp['Systemic Hormonal Preparations, Excl. Sex Hormones And Insulins'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Systemic Hormonal Preparations, Excl. Sex Hormones And Insulins']]
df_atc_generic_name_temp['Various'] = [1 if atc > 0 else atc for atc in df_atc_generic_name_temp['Various']]

Merging with the consolidated data

In [350]:
df_consolidation_data['generic_name'] = df_consolidation_data['generic_name'].str.upper()

In [351]:
df_consolidation_data = pd.merge(df_consolidation_data, df_atc_generic_name_temp, left_on = ['generic_name'], right_on = ['generic_name'], how = 'left')
df_consolidation_data = df_consolidation_data.drop(columns = ['index'])

In [352]:
len(df_consolidation_data)

549813

# Grouping price and quantity

Grouping price and quantity by unique id, labeler, year, and quarter and calculating price per unit adj. for the new data

In [353]:
df_temp = df_consolidation_data.set_index(['unique_id', 'labeler_name', 'year', 'quarter']).groupby(level = ['unique_id', 'labeler_name', 'year', 'quarter'])[['units_reimbursed', 'total_amount_reimbursed_adj']].agg('sum')
df_temp = df_temp.reset_index()

df_temp['price_per_unit_adj'] = df_temp['total_amount_reimbursed_adj'] / df_temp['units_reimbursed']

Dropping the price and quantity and duplicates from the old data

In [354]:
df_consolidation_data = df_consolidation_data.drop(columns = ['units_reimbursed', 'total_amount_reimbursed', 'price_per_unit', 'total_amount_reimbursed_adj', 'price_per_unit_adj'])
df_consolidation_data = df_consolidation_data.drop_duplicates(subset = ['unique_id', 'labeler_name', 'year', 'quarter'])

Merging the org. data

In [355]:
df_consolidation_data = pd.merge(df_consolidation_data, df_temp, left_on = ['unique_id', 'labeler_name', 'year', 'quarter'], right_on = ['unique_id', 'labeler_name', 'year', 'quarter'], how = 'right')

In [356]:
len(df_consolidation_data)

523938

# Downloading raw output

In [357]:
df_consolidation_data.to_csv('raw_consolidation_data.csv', sep = '~', index = False)

# Count

In [358]:
len(df_consolidation_data.drop_duplicates(subset = ['labeler_code', 'product_code'], keep = 'first'))

17625

In [359]:
len(df_consolidation_data.drop_duplicates(subset = ['unique_id'], keep = 'first'))

7214

Executing shutdown due to inactivity...
Executing shutdown due to inactivity...


2022-03-22 21:46:24,825 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...
Executing shutdown...


2022-03-22 21:46:28,936 - INFO     - Executing shutdown...
